In [1]:
import tkinter as tk
import copy
import random

def game_tie(board):
    return all(element != '' for sublst in board for element in sublst)

def check_tris(player, board):
    # Check rows
    for row in board:
        if row.count(player) == 3:
            return True
    # Check columns
    for col in range(len(board[0])):
        if all(board[row][col] == player for row in range(len(board))):
            return True
    # Check diagonals
    if all(board[i][i] == player for i in range(len(board))) or \
       all(board[i][len(board)-1-i] == player for i in range(len(board))):
        return True
    return False

def get_sub_board(board, macro_row, macro_col):
    start_row = macro_row * 3
    start_col = macro_col * 3
    return [[board[start_row + i][start_col + j] 
             for j in range(3)] 
             for i in range(3)]

def get_valid_moves(board, last_move):
    valid_moves = []
    
    if last_move is None:
        # First move - all empty cells are valid
        return [(i, j) for i in range(9) for j in range(9) if board[i][j] == '']
    
    next_board_row = last_move[1] % 3
    next_board_col = last_move[0] % 3
    
    # Check if the next sub-board is full or won
    sub_board = get_sub_board(board, next_board_row, next_board_col)
    if game_tie(sub_board) or check_tris('X', sub_board) or check_tris('O', sub_board):
        # If the targeted sub-board is full/won, allow moves in any empty cell
        return [(i, j) for i in range(9) for j in range(9) if board[i][j] == '']
    
    # Otherwise, only allow moves in the targeted sub-board
    start_row = next_board_row * 3
    start_col = next_board_col * 3
    return [(i, j) for i in range(start_row, start_row + 3) 
            for j in range(start_col, start_col + 3) 
            if board[i][j] == '']

def evaluate_board(board, macro_board):
    if check_tris('X', macro_board):
        return -1000
    elif check_tris('O', macro_board):
        return 1000
    
    score = 0
    # Evaluate each sub-board
    for i in range(3):
        for j in range(3):
            sub_board = get_sub_board(board, i, j)
            if macro_board[i][j] == 'O':
                score += 100
            elif macro_board[i][j] == 'X':
                score -= 100
    return score

def minimax(board, macro_board, depth, last_move, is_maximizing, alpha, beta, max_depth=3):
    if depth == max_depth or check_tris('X', macro_board) or check_tris('O', macro_board) or game_tie(macro_board):
        return evaluate_board(board, macro_board), None

    valid_moves = get_valid_moves(board, last_move)
    if not valid_moves:
        return 0, None

    best_move = random.choice(valid_moves)
    best_value = float('-inf') if is_maximizing else float('inf')

    for move in valid_moves:
        board_copy = copy.deepcopy(board)
        macro_copy = copy.deepcopy(macro_board)
        
        i, j = move
        board_copy[i][j] = 'O' if is_maximizing else 'X'
        
        # Update macro board
        macro_row, macro_col = i // 3, j // 3
        sub_board = get_sub_board(board_copy, macro_row, macro_col)
        if check_tris('O' if is_maximizing else 'X', sub_board):
            macro_copy[macro_row][macro_col] = 'O' if is_maximizing else 'X'
        elif game_tie(sub_board):
            macro_copy[macro_row][macro_col] = '.'
            
        value, _ = minimax(board_copy, macro_copy, depth + 1, (j, i), not is_maximizing, alpha, beta, max_depth)
        
        if is_maximizing:
            if value > best_value:
                best_value = value
                best_move = move
            alpha = max(alpha, best_value)
        else:
            if value < best_value:
                best_value = value
                best_move = move
            beta = min(beta, best_value)
            
        if beta <= alpha:
            break
            
    return best_value, best_move

def update_board(player, row, col):
    board[row][col] = player
    macro_row, macro_col = row // 3, col // 3
    sub_board = get_sub_board(board, macro_row, macro_col)
    
    if check_tris(player, sub_board):
        macro_board[macro_row][macro_col] = player
    elif game_tie(sub_board):
        macro_board[macro_row][macro_col] = '.'

# Board Initialization
board = [['' for _ in range(9)] for _ in range(9)]
macro_board = [['' for _ in range(3)] for _ in range(3)]
last_move = None

def player_move(row, col):
    global last_move
    
    valid_moves = get_valid_moves(board, last_move)
    if (row, col) not in valid_moves or board[row][col] != '':
        return
        
    update_board('X', row, col)
    last_move = (col, row)
    refresh_ui()
    
    if not check_game_over():
        root.after(100, ai_move)  # Short delay before AI move

def ai_move():
    global last_move
    _, move = minimax(board, macro_board, 0, last_move, True, float('-inf'), float('inf'))
    if move:
        update_board('O', move[0], move[1])
        last_move = (move[1], move[0])
        refresh_ui()
        check_game_over()

def check_game_over():
    if check_tris('X', macro_board):
        result_label.config(text="Player X Wins!")
        disable_all_buttons()
        return True
    elif check_tris('O', macro_board):
        result_label.config(text="Player O Wins!")
        disable_all_buttons()
        return True
    elif game_tie(macro_board):
        result_label.config(text="It's a Tie!")
        disable_all_buttons()
        return True
    return False

def disable_all_buttons():
    for i in range(9):
        for j in range(9):
            cells[i][j].config(state=tk.DISABLED)

def refresh_ui():
    valid_moves = get_valid_moves(board, last_move)
    
    for i in range(9):
        for j in range(9):
            cell = cells[i][j]
            cell_text = board[i][j]
            cell.config(text=cell_text)
            
            # Highlight valid moves
            if (i, j) in valid_moves:
                cell.config(bg='lightblue')
            else:
                cell.config(bg='white')

# Initialize Tkinter Window
root = tk.Tk()
root.title("Ultimate Tic-Tac-Toe")

# UI Layout
cells = [[None for _ in range(9)] for _ in range(9)]
for i in range(9):
    for j in range(9):
        cell = tk.Button(root, width=4, height=2, font=("Arial", 16),
                         command=lambda i=i, j=j: player_move(i, j))
        cell.grid(row=i, column=j, padx=1, pady=1)
        if i % 3 == 2:
            cell.grid(padx=1, pady=(1, 10))
        if j % 3 == 2:
            cell.grid(padx=(1, 10), pady=1)
        if i % 3 == 2 and j % 3 == 2:
            cell.grid(padx=(1, 10), pady=(1, 10))
        cells[i][j] = cell

result_label = tk.Label(root, text="", font=("Arial", 20))
result_label.grid(row=9, column=0, columnspan=9)

root.mainloop()